# Mini-projet : classification de battements cardiaques

## Nom du binôme : 

Cet ensemble de données a été créé à l'origine pour un défi d'apprentissage automatique visant à classer les bruits de battements cardiaques. 

<img src="BattementCardiaque.PNG" width="600" height="300"  >

Les données ont été recueillies auprès de deux sources : 
- (A) auprès du grand public via une application de smartphone, 
- (B) dans le cadre d'un essai clinique dans des hôpitaux utilisant le stéthoscope numérique.


Les enregistrements de ces 2 sources étant de durées différentes, ils ont été ensuite transformés en MFCC pour extraire le contenu fréquentiel de ces données. 


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import librosa
import IPython.display as ipd

import matplotlib.pyplot as plt
import sklearn
import math
from scipy import stats
#import seaborn as sns

## Exemples de sons de l'ensemble A de battements cardiaques à classer



In [2]:
# Son d'un battement normal
x0,sr0=librosa.load("normalA.wav",duration=5)
ipd.Audio(x0,rate=sr0)


In [3]:
# Son d'un murmure cardiaque

x1,sr1=librosa.load("murmurA.wav",duration=5)
ipd.Audio(x1,rate=sr1)

In [4]:
# Son d'un artefact
x3,sr3=librosa.load("artifactA.wav",duration=5)
ipd.Audio(x3,rate=sr3)

## Exemples de sons de l'ensemble B de battements cardiaques à classer



In [5]:
# Son d'un battement normal - ensemble B
x4,sr4=librosa.load("normalB.wav",duration=5)
ipd.Audio(x4,rate=sr4)


In [6]:
# Son d'un murmure cardiaque - ensemble B

x5,sr5=librosa.load("murmurB.wav",duration=5)
ipd.Audio(x5,rate=sr5)

## Chargement des données 

A chaque enregistrement, 20 coefficients MFCC sont calculés en réalisant la moyenne sur chaque fenêtre de 10ms.

*DataMFCC.csv* regroupe tous les enregistrements des 2 dispositifs.

Le fichier source *DataMFCC.csv* sur lequel vous travaillez est la conversion des enregistrements audio en matrice de paramètres appélés MFCC (Mel Frequency Cepstral Coefficient) en utilisant la librairie python *librosa*. Ces paramètres permettent d'extraire au mieux le contenu vocal fréquenciel du signal audio.

La matrice de données est composée d'autant de vecteurs lignes que de fichiers audio. Le nombre de colonnes correspond à la dimension du vecteur moyen représentatif des MFCC : ici 20.

In [7]:
data = pd.read_csv('DataMFCC.csv')

data.head()

,mfcc0,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,label
0,-489.62180,70.23919,61.142830,48.898050,36.555330,26.469486,19.623278,15.663741,13.472218,11.869776,...,8.004507,5.803412,3.905093,2.625853,2.056817,2.049077,2.306891,2.535396,2.566249,normal
1,-406.42853,153.23886,-1.369525,16.263828,10.937109,16.939487,4.494656,6.633343,6.228123,3.696192,...,-2.162616,-0.629793,-0.598225,-1.809965,-2.767086,-3.141027,-3.952870,-3.527147,-4.081270,normal
2,-511.58224,82.09152,6.478385,35.782322,4.926917,23.501286,2.172139,16.772097,-0.364136,12.615507,...,10.425499,-2.136471,6.526673,-2.714465,4.534374,-2.864164,2.395745,-2.710135,1.320221,normal
3,-514.13293,78.29218,65.463070,49.305313,34.505836,23.954039,18.073490,15.343872,13.722630,11.918728,...,7.061982,4.627193,2.702706,1.549387,1.235011,1.578198,2.168477,2.570371,2.564390,normal
4,-371.67172,155.28653,35.827470,19.501045,37.935867,34.704395,17.897236,9.181622,10.555704,10.437612,...,-0.750443,2.518559,6.119143,1.359976,-3.627311,-3.266012,0.071095,0.160198,-1.439477,normal


In [8]:
data["label"].value_counts()

normal      351
murmur      129
artifact     40
Name: label, dtype: int64

In [9]:
X=data.iloc[:,0:20]
X.head()

,mfcc0,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19
0,-489.62180,70.23919,61.142830,48.898050,36.555330,26.469486,19.623278,15.663741,13.472218,11.869776,10.107503,8.004507,5.803412,3.905093,2.625853,2.056817,2.049077,2.306891,2.535396,2.566249
1,-406.42853,153.23886,-1.369525,16.263828,10.937109,16.939487,4.494656,6.633343,6.228123,3.696192,-3.058978,-2.162616,-0.629793,-0.598225,-1.809965,-2.767086,-3.141027,-3.952870,-3.527147,-4.081270
2,-511.58224,82.09152,6.478385,35.782322,4.926917,23.501286,2.172139,16.772097,-0.364136,12.615507,-1.403247,10.425499,-2.136471,6.526673,-2.714465,4.534374,-2.864164,2.395745,-2.710135,1.320221
3,-514.13293,78.29218,65.463070,49.305313,34.505836,23.954039,18.073490,15.343872,13.722630,11.918728,9.623236,7.061982,4.627193,2.702706,1.549387,1.235011,1.578198,2.168477,2.570371,2.564390
4,-371.67172,155.28653,35.827470,19.501045,37.935867,34.704395,17.897236,9.181622,10.555704,10.437612,2.700969,-0.750443,2.518559,6.119143,1.359976,-3.627311,-3.266012,0.071095,0.160198,-1.439477


In [10]:
from sklearn.preprocessing import LabelEncoder

# Encode le label de chaque classe par un chiffre
ylabel=data["label"]
le=LabelEncoder().fit(ylabel)
y=le.transform(ylabel)

print('Label',ylabel[40],'correspond au numero',y[40])
print('Label',ylabel[0],'correspond au numero',y[0])
print('Label',ylabel[100],'correspond au numero',y[100])


Label artifact correspond au numero 0
Label normal correspond au numero 2
Label murmur correspond au numero 1


In [11]:
# création de la base d'apprentissage et de test 
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=31,stratify=y)

print("Base d'apprentissage - X_Train: ",len(X_train),"\n","Base de test - X_Test: ",len(X_test),sep="")

Base d'apprentissage - X_Train: 390
Base de test - X_Test: 130


In [12]:
# Description de l'ensemble d'apprentissage
values,count=np.unique(y_train, return_counts=True)

print('label:',values,'nbre de valeurs:',count )

label: [0 1 2] nbre de valeurs: [ 30  97 263]


### Fonctions pour réaliser une table de correspondance entre le label des approches non supervisées et le label réel

In [13]:
def retrieve_info(cluster_labels,y_train):
 #Associe l'étiquette la plus probable à chaque groupe dans le modèle KMeans.
 #Résultats : dictionnaire des clusters associés à chaque étiquette.

# Initialisation
  reference_labels = np.zeros((len(np.unique(cluster_labels)),1))
# Loop pour chaque label 
  for i in range(len(np.unique(cluster_labels))):
    index = np.where(cluster_labels == i,1,0)
    num = np.bincount(y_train[index==1]).argmax()
    reference_labels[i] = num
  return reference_labels

def correspondance(y_pred_kmeans,y_train):
  # Correspondance entre la partition et les classes de la vérité terrain
  reference_labels = retrieve_info(y_pred_kmeans,y_train)
  number_labels = np.zeros(len(y_pred_kmeans))
  for i in range(len(y_pred_kmeans)):
    number_labels[i] = reference_labels[y_pred_kmeans[i]]
  return number_labels

# Méthodes d'apprentissage supervisé 

Appliquer au moins 3 méthodes d'apprentissage supervisé et faire une étude sur les paramètres inhérents 


# Méthode non supervisée


Appliquer au moins une méthode non supervisée

# Votre étude 

Réaliser votre propre étude comparative entre les différents battements cardiaques. 
- jouer sur les bases d'apprentissage/test, équilibre des classes
- les paramètres inhérents aux méthodes
- l'utilisation de prétraitement ou non

Vous pouvez aussi réaliser une étude comparative entre les jeux de données A (*DataMFCC_SetA.csv*) et B (*DataMFCC_SetB.csv*) 
